In [ ]:
%matplotlib inline

from networkx.algorithms import connectivity as conn
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
from tools import utils
import networkx as nx
import time 
import os
import glob
import json

os.chdir("D:\Documenti\Lightning_Network_Logs\\recent_logs")
graphs_list = [None] * len(glob.glob("*json"))
distance_list = [None] * len(glob.glob("*json"))
nodes_size_list = [None] * len(glob.glob("*json"))
edges_size_list = [None] * len(glob.glob("*json"))
avg_degree_list = [None] * len(glob.glob("*json"))
eccentricity_list = [None] * len(glob.glob("*json"))
k_connectivity_list = [None] * len(glob.glob("*json"))
betweenness_centrality_lists = [None] * len(glob.glob("*json"))
# Sets the picture size to a proper dimension.
plt.rcParams['figure.figsize'] = (10, 6)

# Create an array of snapshots of the network
for i, file in enumerate(glob.glob("*.json")):
    with open(file) as f:
        json_graph = json.load(f)
        graph = utils.parse_graph_from_json(json_graph)
        utils.clean_isolated_nodes(graph)
        graph = utils.fix_connectivity(graph)
        graphs_list[i] = graph
        nodes_size_list[i] = nx.number_of_nodes(graph)
        edges_size_list[i] = nx.number_of_edges(graph)
        degree_values = [x[1] for x in graph.degree()]
        sum_of_edges = sum(degree_values) / float(nx.number_of_nodes(graph))
        avg_degree_list[i] = sum_of_edges
        last_graph = graph


In [ ]:
for i, graph in enumerate(graphs_list):
    eccentricity_values = [x[1] for x in nx.eccentricity(graph).items()]
    max_eccentricity = max(eccentricity_values)
    min_eccentricity = min(eccentricity_values)
    eccentricity_list[i] = (max_eccentricity, min_eccentricity)

plt.plot(eccentricity_list)
plt.savefig('eccentricity.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.plot(avg_degree_list)
plt.ylabel('Avg degree')
plt.savefig('avg_degree.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.plot(nodes_size_list)
plt.ylabel('Number of nodes')
plt.savefig('number_of_nodes.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.plot(edges_size_list)
plt.ylabel('Number of edges')
plt.savefig('number_of_edges.png', bbox_inches='tight') 
plt.show()

In [ ]:
for i, graph in enumerate(graphs_list):
    distance_list[i] = nx.diameter(graph)

plt.plot(distance_list)
plt.ylabel('Distance')
plt.savefig('distance.png', bbox_inches='tight')
plt.show()

Calculate the Betweenness Centrality measures for all the snapshot of the network. Returns the subgraph of those network with the most important nodes (BC value greater than 3 times the mean) along with the values returned by the betweenness centality.

Then plots the percentage of important nodes (and their respective edges) wrt the total number of nodes and edges.
**Average duration for 15 snapshots: 5 mins.**

In [ ]:
daily_path = r'D:\Documenti\Lightning_Network_Logs\recent_logs\daily'
os.chdir(daily_path)
daily_list = [None] * len(glob.glob('*.json'))
for i, file in enumerate(glob.glob('*.json')):
    with open(file) as f:
        graph = utils.parse_graph_from_json(json.load(f))
        utils.clean_isolated_nodes(graph)
        graph = utils.fix_connectivity(graph)
        daily_list[i] = nx.betweenness_centrality(graph)

# Sort dictionary of the sums of the betweenness centralities and sort it on their value.
sorted_list = sorted(betweenness_centrality_sums.items(), key=lambda x: x[1], reverse=True)
# Then select the top five nodes of the sorted list.
top_five = [sorted_list[i][0] for i in range(5)]
# For each snapshot of betweenness centrality, select the top 5 nodes and their values.
top_five_list = dict()
for item in top_five:
    top_five_list[item] = [daily_list[i][item] for i in range(len(daily_list))]

plt.rcParams['figure.figsize'] = (10, 6)
plt.xticks(range(number_of_snapshot))
for key in top_five_list.keys():
    plt.plot(top_five_list[key], label=key[0: 5])
plt.legend()
plt.show()

In [ ]:
for i, graph in enumerate(graphs_list):
    betweenness_centrality_lists[i] = nx.betweenness_centrality(graph)


In [ ]:
# Intersection of the nodes that persist for all the snapshots
utils.betweenness_centrality_intersection(betweenness_centrality_lists)
number_of_snapshot = len(betweenness_centrality_lists)

# Finds the most important nodes. The evaluation is performed by aggregating the values for all the nodes and selecting
# the most important ones.
betweenness_centrality_sums = betweenness_centrality_lists[0].copy()
for i in range(1, number_of_snapshot):
    for key in betweenness_centrality_sums.keys():
        betweenness_centrality_sums[key] = betweenness_centrality_sums[key] + betweenness_centrality_lists[i][key]

# Sort dictionary of the sums of the betweenness centralities and sort it on their value.
sorted_list = sorted(betweenness_centrality_sums.items(), key=lambda x: x[1], reverse=True)
# Then select the top five nodes of the sorted list.
top_five = [sorted_list[i][0] for i in range(5)]
# For each snapshot of betweenness centrality, select the top 5 nodes and their values.
top_five_list = dict()
for item in top_five:
    top_five_list[item] = [betweenness_centrality_lists[i][item] for i in range(number_of_snapshot)]

plt.rcParams['figure.figsize'] = (10, 6)
plt.xticks(range(number_of_snapshot))
for key in top_five_list.keys():
    plt.plot(top_five_list[key], label=key[0: 5])
plt.legend()
plt.show()

In [ ]:
import collections as ct

gap_dictionary = ct.defaultdict(list)
# Builds a presence list among the top 15 nodes over the period observed
presence_dict = utils.build_presence_dictionary(betweenness_centrality_lists)
plt.rcParams['figure.figsize'] = (10, 6)

for item in presence_dict:
    start = predecessor = presence_dict[item][0]
    for current in presence_dict[item]:
        if current - predecessor > 1:
            gap = predecessor - start
            gap_dictionary[item].append((start, gap))
            start = current
        predecessor = current
    gap = predecessor - start
    gap_dictionary[item].append((start, gap))

ax = plt.subplot()
ax.set_xlim(0, 15)
ax.set_xticks(range(15))
ax.set_xlabel('Snapshots')
ax.set_yticks(range(len(gap_dictionary.keys())))
ax.set_yticklabels([item[0: 5] for item in gap_dictionary.keys()])
ax.grid(True)
for i, key in enumerate(gap_dictionary.keys()):
    ax.broken_barh(gap_dictionary[key], (i-0.5, 0.8), facecolor='yellow')

Calculates k-vertex connectivity, this task is CPU intensive. Average of 5 mins per snapshot.

In [ ]:
new_path = r'D:\Documenti\Lightning_Network_Logs\recent_logs\k_conn'
utils.removes_satellites(graphs_list[0])
if not os.path.exists(new_path):
    os.makedirs(new_path)
os.chdir(new_path)
start = time.time()
for i, graph in enumerate(graphs_list): 
    k_connectivity_list[i] = conn.k_components(graph)
    end = time.time()
    for key in k_connectivity_list[i].keys():
        for j in range(len(k_connectivity_list[i][key])):
            k_connectivity_list[i][key][j] = list(k_connectivity_list[i][key][j])
    with open('k_conn_' + str(i) + '.json', 'w') as f:
        json.dump(k_connectivity_list[i], f, indent=4)
    elapsed = end - start
    print('Task number ' + str(i) + ' finished in ' + str(elapsed) + ' seconds.')

In [ ]:
# If the k-vertex connectivity has already been evaluated then run this cell
k_conn_path = r'D:\Documenti\Lightning_Network_Logs\recent_logs\k_conn'
k_connectivity_list = utils.read_k_conn_from_files(k_conn_path)

In [ ]:
graph_temp = graphs_list[0].copy()
#sub = graph_temp.subgraph(k_connectivity_list[0]['14'][0]).copy()
#for node in graphs_list[0].nodes:
#    if graph_temp.degree(node) == 1:
#        graph_temp.remove_node(node)
sub = nx.subgraph(graph_temp, k_connectivity_list[0]['14'][0]).copy()

cuts = nx.all_node_cuts(sub)
last_cut = None
for cut in cuts:
    print(cut)
    last_cut = cut

for node in last_cut:
    graph_temp.remove_node(node)
    
print(nx.is_connected(graph_temp))
subgraphs = (graph_temp.subgraph(c) for c in nx.connected_components(graph_temp))
for sub in subgraphs:
    print(sub.order())

In [ ]:
start = time.time()
cutsets = [None] * len(graphs_list)
for i, graph in enumerate(graphs_list): 
    cutsets[i] = nx.all_node_cuts(graph)
    end = time.time()
    elapsed = end - start
    print('Task number ' + str(i) + ' finished in ' + str(elapsed) + ' seconds.')